In [1]:
import carla
import random
import time
import numpy as np
import queue
import cv2



In [2]:
# Connect to the client and retrieve the world object
client = carla.Client('localhost', 2000)
client.set_timeout(2.0)
world = client.get_world()


In [3]:
settings = world.get_settings()
settings.synchronous_mode = True
world.apply_settings(settings)
settings.fixed_delta_seconds = 0.04

In [4]:
# Load layered map for Town 01 with minimum layout
world = client.load_world('Town02_Opt')
world.unload_map_layer(carla.MapLayer.ParkedVehicles)
blueprint_library = world.get_blueprint_library()

In [5]:
world.set_weather(carla.WeatherParameters.ClearNoon)

In [6]:
target_vehicle_bp = blueprint_library.find('vehicle.mercedes.coupe_2020')
agent_vehicle_bp = blueprint_library.find('vehicle.mini.cooper_s')

In [7]:
# 找到所有可以作为初始点的位置并随机选择一个
transform1 = random.choice(world.get_map().get_spawn_points())
transform2 = random.choice(world.get_map().get_spawn_points())

# 在这个位置生成汽车
target_vehicle_bp = world.spawn_actor(target_vehicle_bp, transform1)
agent_vehicle_bp = world.spawn_actor(agent_vehicle_bp, transform2)

# 把它设置成自动驾驶模式
target_vehicle_bp .set_autopilot(True)
agent_vehicle_bp .set_autopilot(True)

In [8]:
rgb_camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
rgb_camera_bp.set_attribute('image_size_x', '1920')
rgb_camera_bp.set_attribute('image_size_y', '1080')
rgb_camera_bp.set_attribute('fov', '110')
# Set the time in seconds between sensor captures
rgb_camera_bp.set_attribute('sensor_tick', '0.1')

In [9]:
depth_camera_bp = world.get_blueprint_library().find('sensor.camera.depth')
depth_camera_bp.set_attribute('image_size_x', '1920')
depth_camera_bp.set_attribute('image_size_y', '1080')
depth_camera_bp.set_attribute('fov', '110')
# Set the time in seconds between sensor captures
depth_camera_bp.set_attribute('sensor_tick', '0.1')

In [10]:
segmentation_camera_bp = world.get_blueprint_library().find('sensor.camera.semantic_segmentation')
segmentation_camera_bp.set_attribute('image_size_x', '1920')
segmentation_camera_bp.set_attribute('image_size_y', '1080')
segmentation_camera_bp.set_attribute('fov', '110')
# Set the time in seconds between sensor captures
segmentation_camera_bp.set_attribute('sensor_tick', '0.1')

In [11]:
camera_transform = carla.Transform(carla.Location(x=1.5, z=2.4))
rgb_camera = world.spawn_actor(rgb_camera_bp, camera_transform, attach_to=agent_vehicle_bp, attachment_type=carla.AttachmentType.Rigid)
image_queue = queue.Queue()
rgb_camera.listen(image_queue.put)


depth_camera = world.spawn_actor(depth_camera_bp, camera_transform, attach_to=agent_vehicle_bp, attachment_type=carla.AttachmentType.Rigid)
image_queue_depth = queue.Queue()
depth_camera.listen(image_queue_depth.put)


segmentation_camera = world.spawn_actor(segmentation_camera_bp, camera_transform, attach_to=agent_vehicle_bp, attachment_type=carla.AttachmentType.Rigid)
image_queue_seg = queue.Queue()
segmentation_camera.listen(image_queue_seg.put)

In [12]:
def get_mask(seg_im, rgb_value):
    # rgb_value should be somethiing like np.uint8([[[70, 70, 70]]])
    # seg_im should be in HSV
    
    hsv_value = cv2.cvtColor(rgb_value, cv2.COLOR_RGB2HSV)
    
    hsv_low = np.array([[[hsv_value[0][0][0]-5, hsv_value[0][0][1], hsv_value[0][0][2]-5]]])
    hsv_high = np.array([[[hsv_value[0][0][0]+5, hsv_value[0][0][1], hsv_value[0][0][2]+5]]])
    
    mask = cv2.inRange(seg_im, hsv_low, hsv_high)
    return mask.any()
object_list = dict()
object_list['car'] = np.uint8([[[ 0, 0, 142]]])


In [13]:
while True:
    world.tick()
    image = image_queue.get()
    #semantic segmentation camera
    image_seg  = image_queue_seg.get()
    #depth camera
    image_depth = image_queue_depth.get()
    # Convert the image to BGR2HSV
    image_seg.convert(carla.ColorConverter.CityScapesPalette)
    img_semseg_hsv = cv2.cvtColor(np.array(image_seg.raw_data).reshape((1080, 1920, 4)), cv2.COLOR_BGR2HSV)
    if get_mask(img_semseg_hsv, object_list['car']):
        image.save_to_disk("/mnt/sda1/carla/fov/%06d.png" %(image.frame))
        image_seg.save_to_disk("/mnt/sda1/carla/seg/%06d_semseg.png" %(image.frame), carla.ColorConverter.CityScapesPalette)
        image_depth.save_to_disk("/mnt/sda1/carla/dep/%06d_depth.png" %(image.frame), carla.ColorConverter.LogarithmicDepth)


: 

: 